<a href="https://colab.research.google.com/github/rutia2044/rutiarbiv-gmail.com/blob/master/VGG_imagnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
 
import torchvision.transforms as transforms
from torchvision.utils import make_grid
 
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
 
import torch.nn as nn
import torch.nn.functional as F
import torch
 
import os
import glob
 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
from tensorflow import keras,summary


In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from keras.applications.vgg16 import VGG16

#from keras import models, layers,applications


In [ ]:
images = []
root = '/content/drive/MyDrive/gan-getting-started/monet_jpg'
for file in tqdm(os.listdir(root)):
    images.append(plt.imread(root+'/'+file))
images = np.array(images)

100%|██████████| 300/300 [00:00<00:00, 413.26it/s]


In [ ]:
images = images/255

In [ ]:
print(images[0][0][0][0])

0.6980392156862745


In [ ]:

#Encoder

EncoderBaseModel = VGG16(include_top=False,input_shape=(256, 256, 3),weights='imagenet')
EncoderBaseModel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [ ]:
ImgPred = EncoderBaseModel.predict(images)
ImgPred.shape

(300, 8, 8, 512)

In [ ]:
from sklearn.manifold import TSNE

#tsneAllImages = TSNE(n_components=700).fit_transform( ImgPred.reshape(300,-1))
tsneAllImages = TSNE(n_components=700,method='exact').fit_transform( ImgPred.reshape(300,-1))
print(tsneAllImages.shape)

(300, 700)


In [ ]:

# save numpy array as csv file
#from numpy import asarray
from numpy import savetxt
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to csv file
savetxt('VGG_TSNE.csv', tsneAllImages, delimiter=',')

In [ ]:

# load numpy array from csv file
from numpy import loadtxt
# load array
tsneAllImages = loadtxt('VGG_TSNE.csv', delimiter=',')
# print the array
print(tsneAllImages)

In [ ]:
v1=ImgPred[0:30,:,:,:]
v2=ImgPred[30:60,:,:,:]
v3=ImgPred[30:60,:,:,:]
v4=ImgPred[30:60,:,:,:]
v5=ImgPred[30:60,:,:,:]
v1.shape

(30, 8, 8, 512)

FID

In [ ]:
# example of calculating the frechet inception distance
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
 
# calculate frechet inception distance
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 


In [ ]:
# define two collections of activations
act1 = random(30*7000)
act1 = act1.reshape((30,7000))
act2 = random(30*7000)
act2 = act2.reshape((30,7000))
# fid between act1 and act1
fid = calculate_fid(act1, act1)
print('FID (same): %.3f' % fid)
# fid between act1 and act2
fid = calculate_fid(act1, act2)
print('FID (different): %.3f' % fid)

FID (same): -0.001
FID (different): 1142.155


In [ ]:
from sklearn.manifold import TSNE

#tsneAllImages = TSNE(n_components=700,method='exact').fit_transform( ImgPred.reshape(300,-1))
tsneAllImages = TSNE(n_components=7000,method='exact').fit_transform( ImgPred.reshape(300,-1))
print(tsneAllImages.shape)


(300, 7000)


In [ ]:
print(tsneAllImages.shape)
tsneV1_Images=tsneAllImages[98:128,:]
tsneV2_Images=tsneAllImages[95:125,:]
selectTSNE=tsneAllImages[90:120,:]

print(tsneV1_Images.shape)
print(tsneV2_Images.shape)
print(selectTSNE.shape)

(300, 700)
(30, 700)
(30, 700)
(30, 700)


In [ ]:
tsneV1_Images = TSNE(n_components=700,method='exact').fit_transform( v1.reshape(30,-1))
#tsneV1_Images = TSNE(n_components=3).fit_transform( v1.reshape(30,-1))
print(tsneV1_Images.shape)
tsneV2_Images = TSNE(n_components=700,method='exact').fit_transform( v2.reshape(30,-1))
#tsneV2_Images = TSNE(n_components=3).fit_transform( v2.reshape(30,-1))
print(tsneV2_Images.shape)

In [ ]:
minIndexesFid=[ 79 ,233 ,252 ,271 , 33, 181, 162 ,142 , 38 ,288, 198 ,227 ,106, 176 ,288 ,163, 148, 282,
 230 ,127  , 4, 227 ,209 , 33 ,223 , 44 ,243, 222, 259 , 80]
aa=np.asarray(minIndexesFid)

In [ ]:
 TSNEminIndexesFid=tsneAllImages[aa,:]
fid = calculate_fid(tsneAllImages, TSNEminIndexesFid)
print('FID (different): %.3f' % fid)
print(TSNEminIndexesFid.shape)

FID (different): 122.463
(30, 700)


In [ ]:
fid = calculate_fid(tsneAllImages, tsneV1_Images)
print('FID (same): %.3f' % fid)

fid = calculate_fid(tsneAllImages, tsneV2_Images)
print('FID (different): %.3f' % fid)

fid = calculate_fid(tsneAllImages, selectTSNE)
print('FID (different): %.3f' % fid)

FID (same): 130.240
FID (different): 153.782
FID (different): 161.074


In [ ]:
#rand 100 vectors of 30 indexs
min=124
for number in range(1000):   
    rndArr=np.random.choice(300, 30)
    selectTSNE=tsneAllImages[rndArr,:]
    fid = calculate_fid(tsneAllImages, selectTSNE)   
    if fid < min:
       print('FID (different): %.3f' % fid)
       min=fid
       minIndx= rndArr
print('Min---FID (different): %.3f' % min)
print(minIndx)

FID (different): 121.143
FID (different): 119.423
FID (different): 117.635


In [ ]:


v1 = v1.reshape((30,32768))
v2 = v2.reshape((30,32768))
fid = calculate_fid(v1, v1)
print('FID (same): %.3f' % fid)

fid = calculate_fid(v1, v2)
print('FID (different): %.3f' % fid)

TSNE + CLUSTERING

In [ ]:
from sklearn.manifold import TSNE


tsneAllImages = TSNE(n_components=2).fit_transform( ImgPred.reshape(300,-1))
tsneAllImages.shape

(300, 2)

In [ ]:
from sklearn.cluster import KMeans
import collections

In [ ]:
kmeans = KMeans(n_clusters=30).fit(tsneAllImages)
collections.Counter(kmeans.labels_)

Counter({0: 1,
         1: 10,
         2: 21,
         3: 2,
         4: 1,
         5: 1,
         6: 83,
         7: 1,
         8: 1,
         9: 1,
         10: 1,
         11: 1,
         12: 1,
         13: 23,
         14: 1,
         15: 1,
         16: 1,
         17: 1,
         18: 1,
         19: 1,
         20: 16,
         21: 19,
         22: 30,
         23: 1,
         24: 1,
         25: 25,
         26: 21,
         27: 17,
         28: 1,
         29: 15})

In [ ]:
select=[]
for i in range (30):
  select.append(np.nonzero(kmeans.labels_ == i)[0][0])
select

[113,
 101,
 2,
 150,
 279,
 254,
 3,
 280,
 32,
 95,
 249,
 157,
 234,
 1,
 115,
 285,
 196,
 175,
 220,
 231,
 24,
 29,
 0,
 215,
 149,
 4,
 9,
 7,
 97,
 22]

In [ ]:
select

[113,
 101,
 2,
 150,
 279,
 254,
 3,
 280,
 32,
 95,
 249,
 157,
 234,
 1,
 115,
 285,
 196,
 175,
 220,
 231,
 24,
 29,
 0,
 215,
 149,
 4,
 9,
 7,
 97,
 22]

In [ ]:
tsneAllImages[select].shape

(30, 700)

AutoEnc Base VGG